### 1. Pandasデータフレームとしてファイルを読み込む<a name="1.1"></a>

In [ ]:
"""
＊重要＊
最初にこのセルを実行してください。
サンプルファイルをダウンロードします。
"""
!wget -q https://raw.githubusercontent.com/CropEvol/lecture/master/data/mutmap_bulk.txt -O mutmap_bulk.txt
    
#--- pandasを読み込む  ---
import pandas as pd

#--- 読み込みファイル名 ---
dataset = 'mutmap_bulk.txt'       

#--- ファイルを読み込む ---
# header=-1（列名行なし）を指定した場合、列名として通し番号が付けられます。
# names=[リスト]とすることで、通し番号の代わりに列名を指定することが可能です。
df = pd.read_csv(dataset, sep="\t", header=-1, names=['chr', 'pos', 'ref_nucl', 'alt_nucl', 'ref_N', 'alt_N'])

#--- 表示 ---
df

### 2. 任意のデータにアクセスする<a name="1.2"></a>

In [ ]:
###### 全て表示（途中のデータは省略されます） ######
df

###### 1列を抽出 ######
#df['ref_nucl']
#df.loc[:, 'ref_nucl']
#df.iloc[:, 2]


###### 複数列を抽出 ######
#df.loc[:, ['ref_nucl','alt_nucl']]
#df.iloc[:, 2:4]


###### 行の抽出 ######
#df.loc[10,:]
#df.iloc[10,:]


###### 複数行を抽出 ######
#df.loc[10:15, :]
#df.iloc[10:15, :]


###### 1セル抽出 ######
#df.loc[10, 'ref_nucl']
#df.iloc[10, 2]


###### 複数列と複数行を抽出 ######
#df.loc[10:15, ['ref_nucl', 'alt_nucl']]
#df.iloc[10:15, 2:4]



### 3. SNP-indexを計算する<a name="1.3"></a>

In [ ]:
###### SNP-indexの計算 ######

# !!! 以下にプログラムを追記する !!!
df['snp_index'] = df['alt_N'] / (df['ref_N'] + df['alt_N'])

#--- 表示 ---
df

### 4. 条件に合うデータを抽出する<a name="1.4"></a>

In [ ]:
###### 条件を指定して抽出 ######

#--- 条件をひとつ指定 ---
# df['ref_nucl']=='A'    # これだけでは望んだ結果が得られない

# df[ df['ref_nucl']=='A' ] 

#--- 複数の条件を指定 ---
# df[ (df['ref_nucl']=='A' ) & (df['alt_nucl']=='G' ) ]    # AND

# df[ (df['ref_nucl']=='A' ) | (df['alt_nucl']=='G' ) ]    # OR


#--- SNP-indexが0.9以上のデータを抽出 ---
# !!! 以下にプログラムを追記する !!!
df[ df['snp_index'] >= 0.9 ]



### 5. ファイルに書き出す<a name="1.5"></a>

In [ ]:
###### ファイルの書き出し ######
#outdata = 'mutmap_snpindex_using_pd.txt'        # 書き出しファイル名
#df.to_csv(outdata, sep='\t', header=True, index=False)

### 6. SNP-indexのグラフを描く<a name="1.6"></a>

In [ ]:
###### SNP-indexのグラフ作成 ######
"""
この1行はmatplotlibを使ったグラフを、Jupyter Notebook上で表示させるために必要です。
Pythonプログラムではありません。Jupyter Notebookの「マジックコマンド」と言われるものです。
"""
%matplotlib inline


"""
ここからPythonプログラム
"""
#--- ライブラリを読み込む ---
import matplotlib.pyplot as plt

#--- x軸データとy軸データ ---
df['snp_index'] = df['alt_N'] / (df['ref_N'] + df['alt_N'])
x = df['pos']
y = df['snp_index']

#--- グラフ描画 ---
fig = plt.figure(figsize=[16,9])    # グラフフィールドの設定
plt.scatter(x, y, color='gray')      # 散布図
plt.title('SNP-index on chromosome 10', fontsize=24)  # グラフタイトル
plt.xlabel('Position (x 10 Mb)', fontsize=16)  # x軸ラベル
plt.ylabel('SNP-index', fontsize=16)           # y軸ラベル


#--- グラフにsnp_index>=0.9のデータを重ね描き ---
# !!! 以下にプログラムを追記する !!!

# snp_index>=0.9のデータを抽出して、新しいデータフレームに入れる
df_ext = df[ df['snp_index'] >= 0.9 ]

x1 = df_ext['pos']       # x軸データ
y1 = df_ext['snp_index'] # y軸データ

plt.scatter(x1, y1, color='red')      # 散布図

### 7. Sliding Window解析<a name="1.7"></a>

In [ ]:
###### Sliding Window解析 ######
#---  NumPyライブラリの読み込み ---
import numpy as np

#--- 区間サイズとステップサイズ ---
CHROM_SIZE = 23207287       # Chromosome 10　の全長 (bp)
WIN_SIZE       = 1 * 1000 * 1000     #  1 Mb = 1000 kb = 1,000,000 bp
STEP_SIZE     = 0.2 * 1000* 1000     #  0.2 Mb = 200 kb = 200,000 bp

#--- 区間データ用のリストを準備 ---
win_position  = []  # 区間中央値用リスト
win_snpindex = []  # 平均SNP-index用リスト

#--- 全ての区間を調べる ---
"""
/// 調べる区間 ///
start, end
0, 0+1000 (kb)
200, 200+1000
400, 400+1000
  .
  .
  .

/// WIN_SIZEとSTEP_SIZEを使って表現すると... ///
繰り返し数:　n = 0, 1, 2, ...

start = STEP_SIZE * n  
end = start + WIN_SIZE


end > CHROM_SIZEになったとき、繰り返しを終える。

このように、繰り返し数を指定しない場合は、for文ではなく、while文を使用します。
"""

n = 0 # 繰り返し数
while True:
    
    #--- 区間のstartとend position ---
    start = STEP_SIZE * n 
    end   = start + WIN_SIZE
    
    #--- 区間の中央値をリストに追加する ---
    p = (start + end) / 2
    win_position.append(p)
    
    #--- 区間内データを抽出 ---
    sub = df[(df['pos'] >= start) & (df['pos'] < end)]
    
    #--- SNP-indexの平均値を算出 ---
    i = sub['snp_index'].mean()
    win_snpindex.append(i)
        
    #--- 繰り返し数を+1 ---
    n += 1
    
    #--- 全ての区間を調べた時、whileから出る ---
    if end > CHROM_SIZE:
        break

#--- グラフ作成  ---
fig = plt.figure(figsize=[16,9])
plt.scatter(x, y, color='gray')      # 全データ
plt.title('SNP-index on chromosome 10', fontsize=24)  # グラフタイトル
plt.xlabel('Position (x 10 Mb)', fontsize=16)  # x軸ラベル
plt.ylabel('SNP-index', fontsize=16)        # y軸ラベル

#--- グラフにsnp_index>=0.9のデータを重ね描き ---
df_ext = df[ df['snp_index'] >= 0.9 ]
x1 = df_ext['pos']
y1 = df_ext['snp_index']
plt.scatter(x1, y1, color='red')

#--- 平均SNP-indexの折れ線グラフ（重ね描き） ---
plt.plot(win_position, win_snpindex, color='blue')      

In [ ]:
### 平均SNP-indexが0.9以上の区間を取り出す ###

#--- Sliding windowのデータをpandasデータフレームにする ---
sw = pd.DataFrame({ 'pos': win_position, 'snp_index': win_snpindex})

#--- 区間のstartとendの位置 ---
sw['start'] = sw['pos'] - WIN_SIZE / 2 
sw['end'] = sw['pos'] + WIN_SIZE / 2

#--- 平均SNP-indexが0.9以上の区間 ---
sw[sw['snp_index'] >= 0.9] 